In [15]:
import json
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import joblib
import time
import sys
sys.path.append("../")

from cfmining.algorithms import MAPOFCEM
from cfmining.criteria import PercentileCalculator, PercentileCriterion, PercentileChangesCriterion, NonDomCriterion
from cfmining.predictors import GeneralClassifier_Shap
from cfmining.action_set import ActionSet
from cfmining.utils import get_data_model

import dice_ml


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
def get_mapofcem_results(
        mapofcem_setter,
        individuals, 
        model, 
        outlier_detection, 
        output_file = None,
    ):
    results = []

    for i in tqdm(range(len(individuals))):
        individual = individuals.iloc[i]
        
        mapofcem = mapofcem_setter(individual.values, model)
        start = time.time()
        mapofcem.fit()
        end = time.time()

        solutions = mapofcem.solutions
        solutions = [s.tolist() for s in solutions]
        is_outlier = sum([outlier_detection.predict(np.array(solution)[None, :]) == -1 for solution in solutions], 0)
        
        results.append({
            "individual" : individual.values.tolist(),
            "prob" : model.predict_proba(individual.values),
            "time" : end - start,
            "n_solutions" : len(mapofcem.solutions),
            "solutions" : solutions,
            "outlier" : is_outlier,
        })

        if output_file is not None:
            pd.DataFrame(results).to_csv(output_file, index=False)

    results = pd.DataFrame(results)
    if output_file is not None:
        results.to_csv(output_file, index=False)
    else:
        return results

In [25]:
def get_dice_results(
    dice_model,
    dice_data,
    individuals,
    mutable_features,
    outlier_detection,
    total_CFs,
    sparsity_weight = 0.2,
    output_file = None,
):
    exp = dice_ml.Dice(dice_data, dice_model)
    results = []
    
    for i in tqdm(range(len(individuals))):
        individual = individuals.iloc[[i]]
        start = time.time()
        
        start = time.time()
        dice_exp = exp.generate_counterfactuals(
            individual,
            total_CFs = total_CFs,
            desired_class = "opposite",
            sparsity_weight = sparsity_weight,
            features_to_vary=mutable_features,
        )
        end = time.time()

        # convert dice solutions to an adequated format
        solutions = json.loads(dice_exp.to_json())["cfs_list"][0]

        # remove last element of all solutions
        solutions = [solution[:-1] for solution in solutions]
        is_outlier = sum([outlier_detection.predict(np.array(solution)[None, :]) == -1 for solution in solutions], 0)

        results.append({
            "individual" : individual.values.tolist(),
            "prob" : dice_model.model.predict_proba(individual.values)[0, 1],
            "time" : end - start,
            "n_solutions" : len(solutions),
            "solutions" : solutions,
            "outlier" : is_outlier,
        })
    results = pd.DataFrame(results)
    if output_file is not None:
        results.to_csv(output_file, index=False)
    else:
        return results

In [37]:
def mapofcem_wrapper(
        action_set,
        criteria,
        criteria_param,
        outlier_detection,
        estimate_outlier,
        total_CFs,
        max_changes
):  
    def f_(ind, model):
        if criteria == "Percentile":
            compare = PercentileCriterion(ind, criteria_param)
        elif criteria == "NonDom":
            compare = NonDomCriterion(criteria_param)
        elif criteria == "PercentileChanges":
            compare = PercentileChangesCriterion(ind, criteria_param)

        return MAPOFCEM(
            action_set,
            ind,
            model,
            outlier_detection=outlier_detection,
            estimate_outlier=estimate_outlier,
            total_CFs=total_CFs,
            max_changes=max_changes,
            compare = compare
        )
    return f_

## German

In [18]:
X_train, Y_train, model, outlier_detection, individuals = get_data_model("german")

In [19]:
action_set = ActionSet(X = X_train)
not_mutable_features = ['Age', 'OwnsHouse', 'isMale', 'JobClassIsSkilled', 'Single', 'ForeignWorker', 'RentsHouse']
mutable_features = [feat for feat in X_train.columns if feat not in not_mutable_features]
continuous_features = ['Age', 'LoanDuration', 'LoanAmount', 'LoanRateAsPercentOfIncome','YearsAtCurrentHome']
for feat in action_set:
    if feat.name in not_mutable_features:
        feat.mutable = False
    if not feat.name in not_mutable_features:
        feat.mutable = True

    feat.step_direction = 0
    feat.update_grid()

In [20]:
extra_not_mutable_features = [
    'LoanRateAsPercentOfIncome', 
    'NumberOfOtherLoansAtBank', 
    'NumberOfLiableIndividuals', 
    'HasTelephone',
    'CheckingAccountBalance_geq_0', 
    'CheckingAccountBalance_geq_200',
    'SavingsAccountBalance_geq_100', 
    'SavingsAccountBalance_geq_500',
    'MissedPayments', 
    'CriticalAccountOrLoansElsewhere', 
    'HasCoapplicant', 
    'HasGuarantor',
]

## mapofcem

In [29]:
percCalc = PercentileCalculator(action_set = action_set)
model_shap = GeneralClassifier_Shap(model, X_train, tree = True)

experiment to fint the best solution

In [38]:
for (i, n_extra_not_mutable) in enumerate([0, 4, 8, 12]):
    if n_extra_not_mutable != 8:
        continue

    extra_not_mut = extra_not_mutable_features[:n_extra_not_mutable]

    not_mut = not_mutable_features + extra_not_mut
    mut = [feat for feat in X_train.columns if feat not in not_mut]
    for feat in action_set:
        if feat.name in not_mut:
            feat.mutable = False
        if not feat.name in not_mut:
            feat.mutable = True

        feat.step_direction = 0
        feat.update_grid()
    

    mapofcem_setter = mapofcem_wrapper(
        action_set=action_set,
        criteria="Percentile",
        criteria_param=percCalc,
        outlier_detection=outlier_detection,
        estimate_outlier=True,
        total_CFs=np.inf,
        max_changes=3
    )

    get_mapofcem_results(
        mapofcem_setter=mapofcem_setter,
        individuals=individuals,
        model=model_shap,
        outlier_detection=outlier_detection,
        output_file=f"../results/german/mapofcem_best_sol_{i}.csv"
    )

  0%|                                                                                                                      | 0/72 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [28:18<00:00, 23.59s/it]


experiment to find solutions in the pareto front

In [ ]:
for (i, n_extra_not_mutable) in enumerate([0]): #, 4, 8, 12]):
    extra_not_mut = extra_not_mutable_features[:n_extra_not_mutable]

    not_mut = not_mutable_features + extra_not_mut
    mut = [feat for feat in X_train.columns if feat not in not_mut]
    for feat in action_set:
        if feat.name in not_mut:
            feat.mutable = False
        if not feat.name in not_mut:
            feat.mutable = True

        feat.step_direction = 0
        feat.update_grid()
    

    mapofcem_setter = mapofcem_wrapper(
        action_set=action_set,
        criteria="PercentileChanges",
        criteria_param=percCalc,
        outlier_detection=outlier_detection,
        estimate_outlier=True,
        total_CFs=np.inf,
        max_changes=4
    )

    get_mapofcem_results(
        mapofcem_setter=mapofcem_setter,
        individuals=individuals,
        model=model_shap,
        outlier_detection=outlier_detection,
        output_file=f"../results/german/mapofcem_pareto_{i}.csv"
    )

## dice

In [27]:
dice_model = dice_ml.Model(
    model = model,
    backend = "sklearn",
    model_type = "classifier"
)
X_train_extended = X_train.copy()
X_train_extended["target"] = Y_train
dice_data = dice_ml.Data(
    dataframe = X_train_extended,
    continuous_features = X_train.columns.tolist(),#continuous_features,
    outcome_name = "target",
)


experiment to find the best solution

In [ ]:
for (i, n_extra_not_mutable) in enumerate([0, 4, 8, 12]):
    extra_not_mut = extra_not_mutable_features[:n_extra_not_mutable]

    not_mut = not_mutable_features + extra_not_mut
    mut = [feat for feat in X_train.columns if feat not in not_mut]
    
    get_dice_results(
        dice_model = dice_model,
        dice_data = dice_data,
        individuals = individuals,
        mutable_features = mut, 
        outlier_detection=outlier_detection,
        total_CFs=1,
        sparsity_weight=0.2,
        output_file=f"../results/german/dice_best_sol_{i}.csv"
    )